In [0]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.dropout = nn.Dropout2d(.2)

        self.conv1_1 = nn.Conv2d(1,16,3)
        self.BN1_1 = nn.BatchNorm2d(16)
        self.conv1_2 = nn.Conv2d(16, 16, 3) 
        self.BN1_2 = nn.BatchNorm2d(16)
        self.conv1_3 = nn.Conv2d(16, 32, 3)
        self.BN1_3 = nn.BatchNorm2d(32)

        self.pool1 = nn.MaxPool2d(2, 2)

        self.conv2_1 = nn.Conv2d(32, 16, 1)
        self.BN2_1 = nn.BatchNorm2d(16)
        self.conv2_2 = nn.Conv2d(16, 16, 3)
        self.BN2_2 = nn.BatchNorm2d(16)
        self.conv2_3 = nn.Conv2d(16, 32, 3)
        self.BN2_3 = nn.BatchNorm2d(32)
        
        #self.pool2 = nn.MaxPool2d(2, 2)
        
        self.Avgpool = nn.AvgPool2d(5) 
        
        self.conv3 = nn.Conv2d(32,10,1)
        
        
    def forward(self, x):
      x = (self.BN1_1(F.relu(self.conv1_1(x)))) #(28-26)3
      x = self.dropout(self.BN1_2((F.relu(self.conv1_2(x))))) #(26-24)5
      x = (self.BN1_3((F.relu(self.conv1_3(x))))) #(24-22)7
      x = self.pool1(x) #(22-11)14
      x = self.dropout(self.BN2_1((F.relu(self.conv2_1(x))))) #(11-11)14
      x = (self.BN2_2((F.relu(self.conv2_2(x))))) #(11-9)16
      x = (self.BN2_3((F.relu(self.conv2_3(x))))) #(9-7)18
      #x = (self.pool2(x)) 
      x = self.Avgpool(x) #
      x = self.conv3(x) 
      x = x.view(-1, 10)
      return F.log_softmax(x)

In [50]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 26, 26]             160
       BatchNorm2d-2           [-1, 16, 26, 26]              32
            Conv2d-3           [-1, 16, 24, 24]           2,320
       BatchNorm2d-4           [-1, 16, 24, 24]              32
         Dropout2d-5           [-1, 16, 24, 24]               0
            Conv2d-6           [-1, 32, 22, 22]           4,640
       BatchNorm2d-7           [-1, 32, 22, 22]              64
         MaxPool2d-8           [-1, 32, 11, 11]               0
            Conv2d-9           [-1, 16, 11, 11]             528
      BatchNorm2d-10           [-1, 16, 11, 11]              32
        Dropout2d-11           [-1, 16, 11, 11]               0
           Conv2d-12             [-1, 16, 9, 9]           2,320
      BatchNorm2d-13             [-1, 16, 9, 9]              32
           Conv2d-14             [-1, 3

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:42: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [0]:


torch.manual_seed(1)
batch_size = 64

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [0]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [0]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1,21):
    print('\nEpoch = {}'.format(epoch))
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/938 [00:00<?, ?it/s]


Epoch = 1


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:35: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  0%|          | 0/938 [00:00<?, ?it/s]


Test set: Average loss: 0.0719, Accuracy: 9812/10000 (98.12%)


Epoch = 2


  0%|          | 0/938 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0176, Accuracy: 9941/10000 (99.41%)


Epoch = 19


  0%|          | 0/938 [00:00<?, ?it/s]


Test set: Average loss: 0.0176, Accuracy: 9945/10000 (99.45%)


Epoch = 20


loss=0.0098065584897995 batch_id=937: 100%|██████████| 938/938 [00:17<00:00, 52.75it/s]



Test set: Average loss: 0.0206, Accuracy: 9928/10000 (99.28%)

